# Homework 4: Functions and Histograms

Please complete this notebook by filling in the cells provided. When you’re done, follow the instructions in [this short explainer video](https://www.youtube.com/watch?v=gMt_Rq43y_4&ab_channel=FahadKamran) to submit your homework.

If you cannot submit online, come to office hours for assistance. The office hours
schedule appears on [data8.org/fa16/weekly.html](http://data8.org/fa16/weekly.html).

This assignment is due Thursday, September 22 at 5PM. You will receive an early submission bonus point if you turn it in by Wednesday, September 21 at 5PM. Directly sharing answers is not okay, but discussing problems with course staff or with other students is encouraged.

Reading:
- Textbook chapters [6](http://www.inferentialthinking.com/chapters/06/visualization.html) and [7](http://www.inferentialthinking.com/chapters/07/functions-and-tables.html)

Run the cell below to prepare the notebook and the tests. **Passing the automatic tests does not guarantee full credit on any question.** The tests are provided to help catch some common errors, but it is *your* responsibility to answer the questions correctly.

In [ ]:
# Run this cell to set up the notebook, but please don't change it.
import numpy as np
from datascience import *

# These lines do some fancy plotting magic.
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import warnings
warnings.simplefilter('ignore', FutureWarning)

from client.api.assignment import load_assignment
tests = load_assignment('hw04.ok')

## 1. Predicting Temperatures


In this exercise, we will try to predict the weather in California using the prediction method  discussed in [section 7.1 of the textbook](https://www.inferentialthinking.com/chapters/07/1/applying-a-function-to-a-column.html).  Much of the code is provided for you; you will be asked to understand and run the code and interpret the results.

The US National Oceanic and Atmospheric Administration (NOAA) operates thousands of climate observation stations (mostly in the US) that collect information about local climate.  Among other things, each station records the highest and lowest observed temperature each day.  These data, called "Quality Controlled Local Climatological Data," are publicly available [here](http://www.ncdc.noaa.gov/orders/qclcd/) and described [here](https://www.ncdc.noaa.gov/data-access/land-based-station-data/land-based-datasets/quality-controlled-local-climatological-data-qclcd).

`temperatures.csv` contains an excerpt of that dataset.  Each row represents a temperature reading in Fahrenheit from one station on one day.  (The temperature is actually the highest temperature observed at that station on that day.)  All the readings are from 2015 and from California stations.

In [56]:
temperatures = Table.read_table("temperatures.csv")
temperatures

Here is a scatter plot:

In [57]:
temperatures.scatter("Date", "Temperature")

Each entry in the column "Date" is a number in MMDD format, meaning that the last two digits denote the day of the month, and the first 1 or 2 digits denote the month.

**Question 1.** Why do the data form vertical bands with gaps?

*Write your answer here, replacing this text.*

Let us solve that problem.  We will convert each date to the number of days since the start of the year.

In [72]:
def get_month(date):
    """The month in the year for a given date.
    
    >>> get_month(315)
    3
    """
    return int(date / 100)

def get_day_in_month(date):
    """The day in the month for a given date.
    
    >>> get_day_in_month(315)
    15
    """
    return date % 100

DAYS_IN_MONTHS = Table().with_columns(
    "Month", np.arange(1, 12+1),
    "Days in Month", make_array(31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31))

# A table with one row for each month.  For each month, we have
# the number of the month (e.g. 3 for March), the number of
# days in that month in 2015 (e.g. 31 for March), and the
# number of days in the year before the first day of that month
# (e.g. 0 for January or 59 for March).
DAYS_SINCE_YEAR_START = DAYS_IN_MONTHS.with_column(
    "Days since start of year", np.cumsum(DAYS_IN_MONTHS.column("Days in Month")) - DAYS_IN_MONTHS.column("Days in Month"))

def days_since_year_start(month):
    """The number of days in the year before this month starts.
    
    month should be the number of a month, like 3 for March.
    
    >>> days_since_year_start(3)
    59
    """
    return DAYS_SINCE_YEAR_START.where("Month", are.equal_to(month))\
                                .column("Days since start of year")\
                                .item(0)

# First, extract the month and day for each reading.
with_month_and_day = temperatures.with_columns(
    "Month", temperatures.apply(get_month, "Date"),
    "Day in month", temperatures.apply(get_day_in_month, "Date"))
# Compute the days-since-year-start for each month and day.
fixed_dates = with_month_and_day.apply(days_since_year_start, "Month") + with_month_and_day.column("Day in month")
# Add those to the table.
with_dates_fixed = with_month_and_day.with_column("Days since start of year", fixed_dates).drop("Month", "Day in month")
with_dates_fixed

**Question 2.** In the cell above, what is the value of this expression?

    np.cumsum(DAYS_IN_MONTHS.column("Days in Month")) - DAYS_IN_MONTHS.column("Days in Month")
    
Describe its type and what its value (or the values in it, if it's an array or table) means.

*Hint:* You can write `np.cumsum?` to get documentation for the function `cumsum`.

In [ ]:
# You can write code in this cell to help you answer this
# question, if you want to.

*Write your answer here, replacing this text.*

Now we can make a better scatter plot.

In [59]:
with_dates_fixed.scatter("Days since start of year", "Temperature")

Let's do some prediction.  For any reading on any day, we will predict its value using all the readings from the week before and after that day.  A reasonable prediction is that the reading will be the average of all those readings.  We will package our code in a function.

In [65]:
PREDICTION_RADIUS = 7

def predict_temperature(day):
    """A prediction of the temperature (in Fahrenheit) on a given day at some station.
    """
    nearby_readings = with_dates_fixed.where("Days since start of year", are.between_or_equal_to(day - PREDICTION_RADIUS, day + PREDICTION_RADIUS))
    return np.average(nearby_readings.column("Temperature"))

**Question 3.** Suppose you're planning a trip to Yosemite for Thanksgiving break this year, and you'd like to predict the temperature on November 26 (the Saturday after Thanksgiving). Use `predict_temperature` to compute a prediction for a temperature reading on that day.

*Hint:* In addition to `predict_temperature`, another function we wrote earlier will be helpful.

In [ ]:
thanksgiving_prediction = ...
thanksgiving_prediction

In [69]:
_ = tests.grade('q1_3')

Below we have computed a predicted temperature for each reading in the table and plotted both.  (It may take a **minute or two** to run the cell.)

In [71]:
with_predictions = with_dates_fixed.with_column(
    "Predicted temperature",
    with_dates_fixed.apply(predict_temperature, "Days since start of year"))
with_predictions.select("Days since start of year", "Temperature", "Predicted temperature")\
                .scatter("Days since start of year")

**Question 4.** How many times was the first line of the body of the function `predict_temperature` (the one that starts with `nearby_readings = ...`) executed when you ran the cell above?

*Write your answer here, replacing this text.*

**Question 5.** The scatter plot is called a *graph of averages*.  In the [example in the textbook](https://www.inferentialthinking.com/chapters/07/1/applying-a-function-to-a-column.html#Example:-Prediction), the graph of averages roughly followed a straight line.  Is that true for this one?  Using your knowledge about the weather, explain why or why not.

*Write your answer here, replacing this text.*

**Question 6.** According to the [Wikipedia article](https://en.wikipedia.org/wiki/Climate_of_California) on California's climate, "[t]he climate of California varies widely, from hot desert to subarctic."  Suppose we limited our data to weather stations in a smaller area whose climate varied less from place to place (for example, the state of Vermont, or the San Francisco Bay Area).  If we made the same graph for that dataset, in what ways would you expect it to look different?

*Write your answer here, replacing this text.*

## 2. Reading Documents


Often, we have to work with data that aren't in CSV format, but instead come in some less nice form.  In this exercise, we'll look at the text of some Reuters news reports from 1987.  Our dataset doesn't include *all* the news reports from that year, but it includes around 20,000 of them.  Reuters doesn't say how they were selected.

We've put the text of all the articles in a file called `reuters.txt`.  The cell below loads that file into a single big string and prints a few thousand characters, which is just enough to see one full article and the start of the next.  (Don't try to print the whole thing, because it's very long.)

In [3]:
# Just run this cell to load the dataset as one big string of text.
with open('reuters.txt', 'r') as file:
    big_reuters_string = file.read()

print("{:.5000}\n[...]".format(big_reuters_string))

There's a bunch of weird text for each article.  Each article is separated from its neighbors by the string `"***ARTICLE***"`.

**Question 1.** Use the String method `split` to make an array of the text of all the articles.  That is, each entry of this array should be the text of one article.  Put that array in a new table called `reuters` as a column with the name "Raw text".

*Hint:* When you split the articles correctly, you should see that each article starts with `"<reuters..."` and ends with `"...</reuters>"`.  There should be 20841 articles.

*Hint 2:* As an example, `"steamcleaner".split("ea")` is the same as `make_array('st', 'mcl', 'ner')`.  So you want to split the big string that contains all the data, splitting with the text `"***ARTICLE***"`.

In [4]:
reuters = Table().with_column(
        ...
    )
reuters

In [4]:
_ = tests.grade('q2_1')

Each article has a line containing its title that looks something like this:

    <title>STANDARD OIL <SRD> TO FORM FINANCIAL UNIT</title>

You could find that yourself for a few of the articles, but it would be very tedious to do it for all 20841 articles.  So we'll write code to do it instead.

**Question 2.** Below, we've written a function called `get_text_in_markers` that will help you find the title text for an article.  Use it to write a function called `get_title`, which is also documented below.

In [5]:
# This function is provided for you to use.  Read at least
# its documentation (the stuff at the beginning in red).
# You can also type in get_text_in_markers? somewhere and
# run it to see the documentation in a slightly nicer form.
# We haven't used any tools you haven't seen yet, so it
# wouldn't hurt to read the code itself, too.
def get_text_in_markers(text, marker):
    """Finds the part of a piece of text that's between specified markers.
    
    Parameters
    ----------
    text : str
        The text in which you want to find something.
    marker : str
        The name of the marker that delimits the part of the
        text you want to grab.  In the text itself, this string
        will be surrounded by "<>" or "</>", but don't include
        those angle brackets in this argument.
    
    Returns
    -------
    str
        The parts of the text that are inside the markers.
    
    Examples
    --------
    >>> get_text_in_markers("stuff <interesting>yay exciting</interesting> more stuff", "interesting")
    'yay exciting'
    """
    start_marker = "<{}>".format(marker)
    end_marker = "</{}>".format(marker)
    split_before = np.array(text.split(start_marker))
    marker_text_and_after = split_before.item(1)
    split_on_end_marker = np.array(marker_text_and_after.split(end_marker))
    return split_on_end_marker.item(0)

# Fill in this function.
def get_title(article_text):
    """Takes the text of an article and returns its title."""
    ...

# When you're done, this should produce 'BAHIA COCOA REVIEW'.
get_title(reuters.column("Raw text").item(0))

In [6]:
_ = tests.grade('q2_2')

**Question 3.** Now use your function to find the title of every article in `reuters`.  Create a new table called `with_titles` that's a copy of `reuters` with an extra column named "Title" that contains these titles.

*Note:* This might take a few seconds to run.

In [6]:
with_titles = ...
with_titles

In [9]:
_ = tests.grade('q2_3')

Now we'll go through a similar process to get the date of each article.  In each article, the date is on its own line, separated from the rest of the article by `<date>` and `</date>` markers.  You can check one of the articles for an example.

**Question 4.** Write a function called `get_date`.  It should take as its argument the whole text of an article and return the date.  The date should be just the day of the year (so January 1 is day 1, and February 1 is day 32, since January has 31 days).  Note that all the articles are from the year 1987, so the year is irrelevant.

We've written a function called `date_string_to_day` that will help you do this.

In [7]:
# This function is provided for you to use.  Read at least
# its documentation (the stuff at the beginning in red).
def date_string_to_day(date_string):
    """Converts a string that looks like a date into the day of the year.
    
    Parameters
    ----------
    date_string : str
        Text that contains a date in any reasonable format.
        For example, "September 13, 1994" or "9/13/94" or
        "13-SEP-1994 15:02:20.00" all work.
    
    Returns
    -------
    int
        The day of the year that the date represents.
    
    Examples
    --------
    >>> date_string_to_mmdd("January 3, 2016")
    3
    
    >>> date_string_to_mmdd("February 4, 2000")
    35
    """
    from dateutil import parser
    date_part = np.array(date_string.strip().split(" ")).item(0)
    try:
        date = parser.parse(date_part) #FIXME
    except:
        print("Failed on", date_string)
    day_in_year = date.timetuple().tm_yday
    return day_in_year

# Fill in this function.
def get_date(article_text):
    date_text = ...
    day_in_year = ...
    ...

# When you're done, this should produce 57.
get_date(reuters.column("Raw text").item(0))

In [11]:
_ = tests.grade('q2_4')

**Question 5.** Use your function to find the date of every article in `with_titles`. Create a new table called `with_dates` that's a copy of `with_titles` with an extra column named "Date" that contains the dates.

In [8]:
with_dates = ...
with_dates

In [13]:
_ = tests.grade('q2_5')

**Question 6.** There was a series of earthquakes in Ecuador on March 6, 1987.  Most Reuters news stories about Ecuador from that period were related to the earthquake or its political and economic consequences.  Find out when Reuters reported on the earthquake by making a histogram of all the dates of the articles whose *titles* include the word `"ECUADOR"`.  Use bins of width 3.

*Hint:* The function `are.containing` creates a predicate that matches strings that contain a given string.  You can find its documentation by running `are.containing?`.

In [10]:
# Use these bins:
bins = np.arange(0, 375, 3)

...

**Question 7.** Make another histogram of the same data, but this time using different bins.  The first bin should start at day 0, and each bin should have a width of 10 days. Then, **using only your own inspection of the histogram (and no other Python code)**, estimate the proportion of Ecuador articles that were reported between days 50 and 100 of the year (including day 50 but not day 100). (The proportion should be out of the total number of articles whose titles include "ECUADOR".) Give that number the name `proportion_50_to_100`.

In [11]:
# Make a histogram as described above.  (Be sure to use the
# bins described in the question.)
...

# By inspecting your histogram, estimate the proportion of
# Ecuador articles that were reported between days 50 and 100
# of the year.  (It's hard to get exactly the right answer
# from a histogram like this, so it's okay if your answer is
# off by a little bit.)
proportion_50_to_100 = ...

**Question 8.** Your histogram should show several long gaps in coverage about Ecuador during the year.  By exploring the dataset, try to explain this.  Use the code cell below for your explorations.

In [ ]:
# Use this cell to answer the question.

*Write your answer here, replacing this text.*

## 3. Causes of Death in California


This exercise is designed to give you practice using the Table method `group`.

We'll be looking at a dataset from the California Department of Public Health (available [here](http://www.healthdata.gov/dataset/leading-causes-death-zip-code-1999-2013) and described [here](http://www.cdph.ca.gov/data/statistics/Pages/DeathProfilesbyZIPCode.aspx)) that records the cause of death (as recorded on a death certificate) for everyone who died in California from 1999 to 2013.  The data are in the file `causes_of_death.csv.zip`.  Each row records the number of deaths by one cause in one year in one ZIP code.

To make the file smaller, we've compressed it; run the next cell to unzip and load it.

In [52]:
!unzip -f causes_of_death.csv.zip
causes = Table.read_table('causes_of_death.csv')
causes

The causes of death in the data are abbreviated.  If you want to know what the abbreviations mean, we've provided a table called `abbreviations.csv`.

**Question 1.** Find the top 5 causes of death in California over the entire period covered by the data.  To do that, create a table with one row for each of the top 5 causes of death, a column called "Cause of Death", and a column called "Count" that records the total number of deaths due to that cause.  Sort it in descending order by count, and call it `top_5_causes`.

In [43]:
# Use this cell to find the top 5 causes of death.

In [54]:
_ = tests.grade('q3_1')

**Question 2.** Create a bar chart that displays the *proportion of all deaths* by each cause.

In [38]:
# Use this cell to make your plot.

**Question 3.** Create a plot of the total number of deaths per year in California.

*Hint:* Use the Table method `plot`.  The first argument is the name or index of the column to put on the horizontal axis.

In [46]:
...

# This line will make the vertical axis start at 0.  You can remove
# it if you want to see the default plot, which is more zoomed-in.
plt.ylim(0, 300000)

**Question 4.** You should see that deaths have increased a little over time, though not uniformly.  How would you explain that?  Describe a dataset you'd like to see to test whether your explanation is valid.

*Write your answer here, replacing this text.*

In [ ]:
# For your convenience, you can run this cell to run all the tests at once!
import os
print("Running all tests...")
_ = [tests.grade(q[:-3]) for q in os.listdir("tests") if q.startswith('q')]
print("Finished running all tests.")